In [1]:
import findspark
findspark.init()

In [2]:
import json,os,sys

In [4]:
import pyspark
from pyspark.sql import functions as F

In [5]:
import logging

logger = logging.getLogger(__name__)

In [6]:
os.getcwd()

'/Users/akshayranade/Documents/GitHub/facctum-dp-data-pipeline-builder/notebooks'

In [7]:
os.chdir('../')

In [8]:
from transform.code_master import CodeMaster

In [9]:
SUBMIT_ARGS = f"--jars /Users/akshayranade/Documents/GitHub/facctum-dp-data-pipeline-builder/jars/*.jar pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS
os.environ['PYSPARK_PYTHON'] = sys.executable

In [10]:
MINIO_ENDPOINT = "http://192.168.1.2:9000/"
MINIO_ACCESS_ID = "minio"
MINIO_SECRET_ID = "minio_minio"
# METASTORE_THRIFT_URI = "thrift://192.168.1.2:9083"

In [11]:
from pyspark.sql import SparkSession, DataFrame
# spark = SparkSession.builder.appName('abc').getOrCreate()

spark = (
    SparkSession.builder.master("local")
    .appName("localTestApp")
    .config("hive.metastore.uris", "thrift://192.168.1.2:9083")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.sql.hive.convertMetastoreParquet", "false")
    .config("spark.sql.hive.caseSensitiveInferenceMode", "NEVER_INFER")
    .config('spark.sql.extensions', 'org.apache.spark.sql.hudi.HoodieSparkSessionExtension')
    .enableHiveSupport()
    .getOrCreate()
)

24/09/21 19:29:59 WARN Utils: Your hostname, Akshays-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
24/09/21 19:29:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/09/21 19:29:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/21 19:30:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/21 19:30:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/09/21 19:30:00 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/09/21 19:30:00 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/09/21 19:30:00 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempt

In [12]:
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", MINIO_ENDPOINT)
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", MINIO_ACCESS_ID)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", MINIO_SECRET_ID)
spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

In [13]:
taxi_zones = spark.read.table("spark_catalog.default.taxi_zones")

24/09/21 19:30:04 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/09/21 19:30:04 WARN DFSPropertiesConfiguration: Cannot find HUDI_CONF_DIR, please set it as the dir of hudi-defaults.conf
24/09/21 19:30:04 WARN DFSPropertiesConfiguration: Properties file file:/etc/hudi/conf/hudi-defaults.conf not found. Ignoring to load props file


In [14]:
taxi_zones.show(20)

+-------------------+--------------------+--------------------+----------------------+--------------------+----------+-------------+--------------------+------------+-------------------+--------------------+-----------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|locationid|      borough|                zone|service_zone|        business_ts|                 _id|business_dt|
+-------------------+--------------------+--------------------+----------------------+--------------------+----------+-------------+--------------------+------------+-------------------+--------------------+-----------+
|  20240916082355921|20240916082355921...|4b254c72-836c-4b3...|  business_dt=2024-...|6409ce3c-194e-4df...|         1|          EWR|      Newark Airport|         EWR|2024-09-16 08:23:48|4b254c72-836c-4b3...| 2024-09-04|
|  20240916082355921|20240916082355921...|aa215441-fe4f-45f...|  business_dt=2024-...|6409ce3c-194e-4df...|         2|  

24/09/21 19:30:14 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [10]:
# employee_all = spark.read.parquet('/Users/akshayranade/Documents/Personal/spark_output/pipeline_demo/employee_all')

In [11]:
# employee_all.show(10)

In [12]:
# Postgres 

url = 'jdbc:postgresql://localhost:5436/ofti_test'
properties = {'user': 'postgres', 'password': 'postgres', "driver": "org.postgresql.Driver"}

In [13]:
# df = spark.read.jdbc(url=url, table='tablename', properties=properties)

In [14]:
input_data={}

In [15]:
simpleData = [("James", "Sales", 3000),
        ("Michael", "Sales", 4600),
        ("Robert", "Sales", 4100),
        ("Maria", "Finance", 3000),
        ("James", "Sales", 3000),
        ("Scott", "Finance", 3300),
        ("Jen", "Finance", 3900),
        ("Jeff", "Marketing", 3000),
        ("Kumar", "Marketing", 2000),
        ("Saif", "Sales", 4100)
]
schema = ["employee_name", "department", "salary"]

employee_department = spark.createDataFrame(data=simpleData, schema = schema)

#Add to input_data dictionary
input_data['employee_department'] = employee_department

simpleData = [("James", "Stewart", 11111),
    ("Michael", "Paine", 44444),
    ("Robert", "Baggins", 12345),
    ("Maria", "Hanks", 34567),
    ("James", "Snape", 43215),
    ("Scott", "Potter", 65234),
    ("Jen", "Granger", 76234),
    ("Jeff", "Weasley", 54234),
    ("Kumar", "Oppenheimer", 34567),
    ("Saif", "Tesla", 43215)
]
schema = ["first_name", "last_name", "pin_code"]

employee_demo = spark.createDataFrame(data=simpleData, schema = schema)

#Add to input_data dictionary
input_data['employee_demo'] = employee_demo

In [16]:
employee_department.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



24/09/16 12:31:50 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [13]:
# employee_department.coalesce(1).write.parquet('/Users/akshayranade/Documents/Personal/spark_output/pipeline_demo/employee_department')

In [17]:
# for column_name in employee_department.columns:
#     employee_department=employee_department.withColumnRenamed(column_name,column_name+"_left")

In [18]:
# employee_department.show(20)

24/08/28 16:57:44 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [19]:
# for column_name in employee_department.columns:
#     employee_department=employee_department.withColumnRenamed(column_name,column_name.replace("_left",""))

In [17]:
employee_department.show(20)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



In [18]:
#Write to postgres
employee_department.write.jdbc(url=url, table="employee_department", mode='overwrite', properties=properties)

24/09/16 16:58:28 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 122544 ms exceeds timeout 120000 ms
24/09/16 16:58:28 WARN SparkContext: Killing executors is not supported by current scheduler.
24/09/16 16:58:31 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [16]:
employee_department = spark.read.jdbc(url=url, table='public.employee_department', properties=properties)

In [17]:
employee_department.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



In [18]:
employee_department.show(10)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



24/08/29 21:45:48 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 943834 ms exceeds timeout 120000 ms
24/08/29 21:45:48 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/29 22:16:47 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [24]:
employee_demo.write.mode('overwrite').parquet('/Users/akshayranade/Documents/Personal/spark_output/pipeline_demo/employee_demo')

In [ ]:
# xyz.write.parquet('/Users/akshayranade/Documents/Personal/spark_output/xyz')

In [ ]:
# auto sales data from https://www.carpro.com/blog/full-year-2022-national-auto-sales-by-brand
# data = [('Toyota', 1849751, -9), ('Ford', 1767439, -2), ('Chevrolet', 1502389, 6),
#         ('Honda', 881201, -33), ('Hyundai', 724265, -2), ('Kia', 693549, -1),
#         ('Jeep', 684612, -12), ('Nissan', 682731, -25), ('Subaru', 556581, -5),
#         ('Ram Trucks', 545194, -16), ('GMC', 517649, 7), ('Mercedes-Benz', 350949, 7),
#         ('BMW', 332388, -1), ('Volkswagen', 301069, -20), ('Mazda', 294908, -11),
#         ('Lexus', 258704, -15), ('Dodge', 190793, -12), ('Audi', 186875, -5),
#         ('Cadillac', 134726, 14), ('Chrysler', 112713, -2), ('Buick', 103519, -42),
#         ('Acura', 102306, -35), ('Volvo', 102038, -16), ('Mitsubishi', 102037, -16),
#         ('Lincoln', 83486, -4), ('Porsche', 70065, 0), ('Genesis', 56410, 14),
#         ('INFINITI', 46619, -20), ('MINI', 29504, -1), ('Alfa Romeo', 12845, -30),
#         ('Maserati', 6413, -10), ('Bentley', 3975, 0), ('Lamborghini', 3134, 3),
#         ('Fiat', 915, -61), ('McLaren', 840, -35), ('Rolls-Royce', 460, 7)]

# dq = spark.createDataFrame(data, ["Brand", "US_Sales_2022", "Sales_Change_Percentage"])

In [ ]:
# dq.show(20)

In [ ]:
col_name = 'col1_left'

In [ ]:
col_name.rstrip("_left")

In [ ]:
# data = {}

In [ ]:
# data['xyz'] = xyz
# data['dq'] = dq

In [25]:
input_data

{'employee_department': DataFrame[employee_name: string, department: string, salary: bigint],
 'employee_demo': DataFrame[first_name: string, last_name: string, pin_code: bigint]}

In [14]:
f = open ('../conf/source.json', "r")
source_conf = json.loads(f.read())

In [15]:
source_conf

{'project_name': 'ar',
 'version': '0.1',
 'input_tables': {'postgres': {},
  'object': {'employee_demo': '/Users/akshayranade/Documents/Personal/spark_output/pipeline_demo/employee_demo',
   'employee_department': '/Users/akshayranade/Documents/Personal/spark_output/pipeline_demo/employee_department'}},
 'output_tables': {'postgres': {},
  'minio': {},
  'object': {'stats_by_department': '/Users/akshayranade/Documents/Personal/spark_output/pipeline_demo/stats_by_department'}},
 'db_conf': {'postgres': {'url': 'jdbc:postgresql://localhost:5432/ofti_test',
   'user': 'postgres',
   'password': 'postgres'},
  'object': {},
  'minio': {'url': 'http://172.16.10.138:9000/',
   'user': 'minio',
   'password': 'minio_minio'}}}

In [18]:
bool(source_conf['input_tables'].get('postgres'))

False

24/08/30 20:00:42 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 296012 ms exceeds timeout 120000 ms
24/08/30 20:00:42 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/30 20:00:44 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [34]:
input_data = {}
for data_source in source_conf["input_tables"].keys():
    print(data_source)
    if 'postgres' in data_source:
        db_url = source_conf['db_conf'][data_source]['url']
        db_properties = {'user': source_conf['db_conf'][data_source]['user'],
                        "password": source_conf['db_conf'][data_source]['password'],
                        "driver": "org.postgresql.Driver"}
        
        for input_table in source_conf["input_tables"].get(data_source):
            print(input_table)
            df = spark.read.jdbc(url=db_url, table=source_conf["input_tables"][data_source].get(input_table), properties=db_properties)
            input_data[input_table] = df

postgres
employee_department
object


In [35]:
input_data

{'employee_department': DataFrame[employee_name: string, department: string, salary: bigint]}

In [36]:
input_data['employee_department'].show(10)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



24/08/28 18:01:33 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 863540 ms exceeds timeout 120000 ms
24/08/28 18:01:33 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/28 18:01:38 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [28]:
f = open ('../conf/pipeline.json', "r")
pipeline_dict = json.loads(f.read())

In [29]:
pipeline_dict

{'module_name': 'ar',
 'version': '0.1',
 'pipeline': {'stage_1': {'join': {'left_table': 'employee_department',
    'right_table': 'employee_demo',
    'join_condition': ['employee_name_left == first_name'],
    'join_type': 'left',
    'output': 'employee_all'}},
  'stage_2': {'aggregate': {'table_name': 'employee_all',
    'group_by_cols': ['department'],
    'aggregate_functions': {'salary': 'sum', 'employee_name': 'count'},
    'output': 'stats_by_department'}},
  'stage_3': {'aggregate': {'table_name': 'employee_all',
    'group_by_cols': ['pin_code'],
    'aggregate_functions': {'salary': 'sum'},
    'output': 'income_by_region'}}}}

In [28]:
#code master

code_master_obj = CodeMaster(pipeline=pipeline_dict, data=input_data, spark=spark)

In [29]:
df, df_dict = code_master_obj.build_pipeline()

In [30]:
df_dict

{'employee_all': DataFrame[employee_name: string, department: string, salary: bigint, first_name: string, last_name: string, pin_code: bigint],
 'stats_by_department': DataFrame[department: string, count_employee_name: bigint, sum_salary: bigint],
 'income_by_region': DataFrame[pin_code: bigint, sum_salary: bigint]}

In [31]:
df_dict['employee_all'].show(20)

+-------------+----------+------+----------+-----------+--------+
|employee_name|department|salary|first_name|  last_name|pin_code|
+-------------+----------+------+----------+-----------+--------+
|        Scott|   Finance|  3300|     Scott|     Potter|   65234|
|        James|     Sales|  3000|     James|      Snape|   43215|
|        James|     Sales|  3000|     James|    Stewart|   11111|
|        James|     Sales|  3000|     James|      Snape|   43215|
|        James|     Sales|  3000|     James|    Stewart|   11111|
|          Jen|   Finance|  3900|       Jen|    Granger|   76234|
|      Michael|     Sales|  4600|   Michael|      Paine|   44444|
|        Kumar| Marketing|  2000|     Kumar|Oppenheimer|   34567|
|         Saif|     Sales|  4100|      Saif|      Tesla|   43215|
|        Maria|   Finance|  3000|     Maria|      Hanks|   34567|
|       Robert|     Sales|  4100|    Robert|    Baggins|   12345|
|         Jeff| Marketing|  3000|      Jeff|    Weasley|   54234|
+---------

In [32]:
df_dict['stats_by_department'].show(20)

+----------+-------------------+----------+
|department|count_employee_name|sum_salary|
+----------+-------------------+----------+
|     Sales|                  7|     24800|
|   Finance|                  3|     10200|
| Marketing|                  2|      5000|
+----------+-------------------+----------+



In [33]:
df_dict['income_by_region'].show(20)

+--------+----------+
|pin_code|sum_salary|
+--------+----------+
|   65234|      3300|
|   54234|      3000|
|   12345|      4100|
|   43215|     10100|
|   11111|      6000|
|   76234|      3900|
|   44444|      4600|
|   34567|      5000|
+--------+----------+



In [34]:
df_dict['employee_all'].write.jdbc(url=url, table="employee_all", mode='overwrite', properties=properties)
df_dict['stats_by_department'].write.jdbc(url=url, table="stats_by_department", mode='overwrite', properties=properties)
df_dict['income_by_region'].write.jdbc(url=url, table="income_by_region", mode='overwrite', properties=properties)

In [ ]:
pipeline = pipeline_dict['pipeline']

In [ ]:
for stage in pipeline.keys():
    print (stage)
    print(pipeline.get(stage))
    stage_operations = pipeline.get(stage)
    for oper in stage_operations.keys():
        print(oper)
        print(stage_operations.get(oper))

In [ ]:
sel_details = pipeline['pipeline']['stage_1']['select']
sel_details

In [ ]:
df1 = selection(sel_details)

In [ ]:
df1.show(20)

In [ ]:
def selection(details:dict):
    try:
        df_sel = data[details["table_name"]]
    except:
        logger.error("No dataframe mentioned to select.")
        exit()

    if 'cols_to_select' in details.keys():
        try:
            df_sel = df_sel.select(*details["cols_to_select"])
        except:
            logger.info('Column selection %s for table %s is incorrect.', str(details["cols_to_select"]), details["table_name"])
            exit()
    
    if 'drop' in details.keys():
        df_sel = df_sel.drop(*details["drop"])

    if 'dedup' in details.keys():
        if details['dedup'].upper() == "TRUE":
            df_sel = df_sel.drop_duplicates()

    return df_sel

In [1]:
import pandas as pd

In [2]:
taxi_trips = pd.read_csv('/Users/akshayranade/Documents/Ooredoo_data_platform/playground2/datafiles/taxi_trips_2020.csv', low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/akshayranade/Documents/Ooredoo_data_platform/playground2/datafiles/taxi_trips_2020.csv'

In [28]:
taxi_trips.dtypes

VendorID                 float64
lpep_pickup_datetime      object
lpep_dropoff_datetime     object
store_and_fwd_flag        object
RatecodeID               float64
PULocationID               int64
DOLocationID               int64
passenger_count          float64
trip_distance            float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
payment_type             float64
trip_type                float64
congestion_surcharge     float64
dtype: object

In [29]:
taxi_trips.head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2020-01-01 06:47:28.000,2020-01-01 06:52:54.000,N,1.0,74,75,1.0,1.47,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
1,2.0,2020-01-01 13:25:34.000,2020-01-01 13:30:43.000,N,1.0,74,75,1.0,1.49,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
2,2.0,2020-01-01 14:20:35.000,2020-01-01 14:26:25.000,N,1.0,74,75,1.0,1.31,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
3,2.0,2020-01-02 06:56:47.000,2020-01-02 07:03:03.000,N,1.0,74,75,1.0,1.43,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
4,2.0,2020-01-02 09:34:46.000,2020-01-02 09:41:02.000,N,1.0,74,75,1.0,1.10,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
5,2.0,2020-01-02 09:37:42.000,2020-01-02 09:44:34.000,N,1.0,74,75,1.0,1.24,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
6,2.0,2020-01-02 10:23:34.000,2020-01-02 10:29:57.000,N,1.0,74,75,1.0,1.33,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
7,2.0,2020-01-03 09:41:28.000,2020-01-03 09:48:02.000,N,1.0,74,75,1.0,1.35,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
8,2.0,2020-01-03 09:02:30.000,2020-01-03 09:10:00.000,N,1.0,74,75,1.0,1.19,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0
9,2.0,2020-01-03 10:43:08.000,2020-01-03 10:48:46.000,N,1.0,74,75,1.0,1.35,6.5,0.0,0.5,0.0,0.0,0.3,7.3,2.0,1.0,0.0


In [30]:
taxi_trips.to_csv('/Users/akshayranade/Documents/Ooredoo_data_platform/playground2/datafiles/taxi_trips_2020.csv', index=False)

In [3]:
#csv file name to be read in 
in_csv = '/Users/akshayranade/Downloads/NYC_Taxi_Trips/taxi_trips/2017_taxi_trips.csv'

In [4]:
tt = pd.read_csv(in_csv, low_memory=False)

In [5]:
tt.shape[0]

11740667

In [188]:
1734051 + 6044050 + 8807303 + 11740667

28326071

In [6]:
tt.dtypes

VendorID                   int64
lpep_pickup_datetime      object
lpep_dropoff_datetime     object
store_and_fwd_flag        object
RatecodeID                 int64
PULocationID               int64
DOLocationID               int64
passenger_count            int64
trip_distance            float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
payment_type               int64
trip_type                float64
dtype: object

In [7]:
tt.head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type
0,1,2017-01-01 09:00:01.000,2017-01-01 09:03:56.000,N,1,74,41,1,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2,1.0
1,1,2017-01-01 18:57:55.000,2017-01-01 19:01:16.000,N,1,42,41,1,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2,1.0
2,2,2017-01-02 06:55:47.000,2017-01-02 06:58:54.000,N,1,42,41,1,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2,1.0
3,2,2017-01-02 14:34:17.000,2017-01-02 14:39:29.000,N,1,74,41,1,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2,1.0
4,1,2017-01-03 06:12:30.000,2017-01-03 06:15:56.000,N,1,42,41,1,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2,1.0
5,2,2017-01-03 10:52:24.000,2017-01-03 10:56:35.000,N,1,41,41,1,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2,1.0
6,2,2017-01-03 10:56:13.000,2017-01-03 11:00:24.000,N,1,75,41,1,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2,1.0
7,1,2017-01-05 12:32:51.000,2017-01-05 12:37:11.000,N,1,42,41,1,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2,1.0
8,1,2017-01-05 14:23:16.000,2017-01-05 14:27:18.000,N,1,152,41,1,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2,1.0
9,2,2017-01-06 07:39:25.000,2017-01-06 07:44:19.000,N,1,42,41,1,0.7,5.0,0.0,0.5,0.0,0.0,0.3,5.8,2,1.0


In [81]:
tt_2019['lpep_pickup_datetime'] = pd.to_datetime(tt_2019['lpep_pickup_datetime'], infer_datetime_format=True)
tt_2019['lpep_dropoff_datetime'] = pd.to_datetime(tt_2019['lpep_dropoff_datetime'], infer_datetime_format=True)

/var/folders/vl/bqtd7jyn4clbfdzrdsv6llxh0000gp/T/ipykernel_73236/1826201379.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  tt_2019['lpep_pickup_datetime'] = pd.to_datetime(tt_2019['lpep_pickup_datetime'], infer_datetime_format=True)
/var/folders/vl/bqtd7jyn4clbfdzrdsv6llxh0000gp/T/ipykernel_73236/1826201379.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  tt_2019['lpep_dropoff_datetime'] = pd.to_datetime(tt_2019['lpep_dropoff_datetime'], infer_datetime_format=True)


In [82]:
tt_2019['lpep_pickup_datetime'] = tt_2019['lpep_pickup_datetime'].dt.strftime('%Y%m%d:%H:%M:%S')
tt_2019['lpep_dropoff_datetime'] = tt_2019['lpep_dropoff_datetime'].dt.strftime('%Y%m%d:%H:%M:%S')



In [83]:
tt_2019.head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,20190417:08:58:00,20190417:09:04:30,N,1.0,41,42,1.0,1.20,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
1,2.0,20190417:12:36:53,20190417:12:43:34,N,1.0,41,42,1.0,1.12,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
2,2.0,20190417:12:17:55,20190417:12:25:18,N,1.0,41,42,1.0,1.28,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
3,2.0,20190417:15:49:50,20190417:15:58:01,N,1.0,41,42,1.0,1.03,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
4,2.0,20190418:13:22:23,20190418:13:29:37,N,1.0,41,42,1.0,1.35,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
5,2.0,20190418:14:44:08,20190418:14:52:16,N,1.0,41,42,1.0,1.10,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
6,2.0,20190419:08:43:45,20190419:08:50:53,N,1.0,41,42,1.0,1.13,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
7,2.0,20190419:11:47:12,20190419:11:55:50,N,1.0,41,42,1.0,0.96,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
8,2.0,20190419:12:18:01,20190419:12:25:49,N,1.0,41,42,1.0,1.35,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
9,2.0,20190419:12:02:40,20190419:12:10:40,N,1.0,41,42,1.0,0.84,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0


In [96]:
tt_2019[0:20]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,20190417:08:58:00,20190417:09:04:30,N,1.0,41,42,1.0,1.20,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
1,2.0,20190417:12:36:53,20190417:12:43:34,N,1.0,41,42,1.0,1.12,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
2,2.0,20190417:12:17:55,20190417:12:25:18,N,1.0,41,42,1.0,1.28,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
3,2.0,20190417:15:49:50,20190417:15:58:01,N,1.0,41,42,1.0,1.03,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
4,2.0,20190418:13:22:23,20190418:13:29:37,N,1.0,41,42,1.0,1.35,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
5,2.0,20190418:14:44:08,20190418:14:52:16,N,1.0,41,42,1.0,1.10,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
6,2.0,20190419:08:43:45,20190419:08:50:53,N,1.0,41,42,1.0,1.13,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
7,2.0,20190419:11:47:12,20190419:11:55:50,N,1.0,41,42,1.0,0.96,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
8,2.0,20190419:12:18:01,20190419:12:25:49,N,1.0,41,42,1.0,1.35,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0
9,2.0,20190419:12:02:40,20190419:12:10:40,N,1.0,41,42,1.0,0.84,7.0,0.0,0.5,0.0,0.0,0.3,7.8,2.0,1.0,0.0


In [84]:
tt_2019.dtypes

VendorID                 float64
lpep_pickup_datetime      object
lpep_dropoff_datetime     object
store_and_fwd_flag        object
RatecodeID               float64
PULocationID               int64
DOLocationID               int64
passenger_count          float64
trip_distance            float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
payment_type             float64
trip_type                float64
congestion_surcharge     float64
dtype: object

In [38]:
#csv file name to be read in 
in_csv = '/Users/akshayranade/Downloads/NYC_Taxi_Trips/taxi_trips/2017_taxi_trips.csv'

In [39]:
full_df = pd.read_csv(in_csv, low_memory=False)

In [40]:
full_df.shape[0]

11740667

In [41]:
#get the number of lines of the csv file to be read
number_lines = sum(1 for row in (open(in_csv)))

In [42]:
number_lines

11740668

In [45]:
#size of rows of data to write to the csv, 
rowsize = number_lines//12

In [46]:
rowsize

978389

In [47]:
#start looping through data writing it to a new file for each set
count = 0

df1 = pd.read_csv(in_csv, low_memory=False)

for i in range(0,number_lines,rowsize):
    #skip rows that have been read
    print(i)
    df = df1[i:i+rowsize]
    count = count + df.shape[0]
    
    # df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'], infer_datetime_format=True)
    # df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'], infer_datetime_format=True)

    # df['lpep_pickup_datetime'] = df['lpep_pickup_datetime'].dt.strftime('%Y%m%d:%H:%M:%S')
    # df['lpep_dropoff_datetime'] = df['lpep_dropoff_datetime'].dt.strftime('%Y%m%d:%H:%M:%S')

    #csv to write data to a new file with indexed name. input_1.csv etc.
    out_csv = '/Users/akshayranade/Documents/Ooredoo_data_platform/playground2/datafiles/taxi_trips_2017_' + str(i) + '.csv'
    df.to_csv(out_csv,
          index=False,
          mode='a',#append data to csv file
          chunksize=rowsize)#size of data to append for each loop

0
978389
1956778
2935167
3913556
4891945
5870334
6848723
7827112
8805501
9783890
10762279


In [50]:
df.head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type
10762279,2,2017-10-26 15:49:23.000,2017-10-26 15:52:59.000,N,1,7,7,1,0.58,4.5,0.0,0.5,0.0,0.0,0.3,5.3,2,1.0
10762280,2,2017-10-26 15:00:26.000,2017-10-26 15:04:14.000,N,1,7,7,1,0.54,4.5,0.0,0.5,0.0,0.0,0.3,5.3,2,1.0
10762281,2,2017-10-26 15:14:48.000,2017-10-26 15:18:42.000,N,1,7,7,1,0.78,4.5,0.0,0.5,0.0,0.0,0.3,5.3,2,1.0
10762282,2,2017-10-27 09:53:36.000,2017-10-27 09:56:42.000,N,1,7,7,1,0.53,4.5,0.0,0.5,0.0,0.0,0.3,5.3,2,1.0
10762283,2,2017-10-27 09:28:32.000,2017-10-27 09:32:39.000,N,1,7,7,1,0.57,4.5,0.0,0.5,0.0,0.0,0.3,5.3,2,1.0
10762284,2,2017-10-27 09:54:39.000,2017-10-27 09:57:32.000,N,1,7,7,1,0.56,4.5,0.0,0.5,0.0,0.0,0.3,5.3,2,1.0
10762285,2,2017-10-27 10:24:44.000,2017-10-27 10:28:41.000,N,1,7,7,1,0.62,4.5,0.0,0.5,0.0,0.0,0.3,5.3,2,1.0
10762286,2,2017-10-27 13:43:11.000,2017-10-27 13:47:10.000,N,1,7,7,1,0.69,4.5,0.0,0.5,0.0,0.0,0.3,5.3,2,1.0
10762287,2,2017-10-27 14:16:19.000,2017-10-27 14:19:48.000,N,1,7,7,1,0.71,4.5,0.0,0.5,0.0,0.0,0.3,5.3,2,1.0
10762288,2,2017-10-27 15:02:47.000,2017-10-27 15:06:35.000,N,1,7,7,1,0.74,4.5,0.0,0.5,0.0,0.0,0.3,5.3,2,1.0


In [48]:
count

11740667

In [49]:
count - 11740667

0